In [257]:
import numpy as np
import pandas as pd
import networkx as nx
import random
from random import sample
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib as mpl
import ast
from scipy import spatial
import csv
from tqdm.notebook import tqdm
import re
%matplotlib inline

In [258]:
df_total = pd.read_excel('data/all_data.xlsx', sheet_name='percentChange',engine='openpyxl')
stocks = pd.read_csv('data/stocks.csv',)
links_total = pd.read_csv('data/links_total.csv',)

In [259]:
all_sectors = {}
for sect in df_total['Sector']:
    if(sect in all_sectors):
        all_sectors[sect]+=1
    else:
        all_sectors[sect]=1
sorted_sectors = sorted(all_sectors.items(),key=lambda x:x[1],reverse=True)

### Identifying Popular Sectors
The snippet below prints all the sectors from our raw data, these include banking, software, pharmaceuticals and many more.
With these sectors now identified, we can begin exploring the relationship between them, and see how stocks within the same sectors relate to eachother.

In [260]:
for sector in sorted_sectors:
    print(sector)

(nan, 375)
('Banks', 246)
('Biotechnology', 126)
('Oil, Gas & Consumable Fuels', 122)
('Equity Real Estate Investment Trusts ...', 92)
('Metals & Mining', 89)
('Machinery', 85)
('Health Care Equipment & Supplies', 83)
('Electronic Equipment, Instruments & C...', 78)
('Insurance', 75)
('Capital Markets', 75)
('Semiconductors & Semiconductor Equipment', 68)
('Software', 66)
('Specialty Retail', 58)
('Commercial Services & Supplies', 52)
('IT Services', 52)
('Chemicals', 51)
('Hotels, Restaurants & Leisure', 50)
('Pharmaceuticals', 48)
('Thrifts & Mortgage Finance', 47)
('Health Care Providers & Services', 46)
('Communications Equipment', 45)
('Food Products', 44)
('Media', 42)
('Household Durables', 39)
('Aerospace & Defense', 37)
('Professional Services', 35)
('Energy Equipment & Services', 32)
('Textiles, Apparel & Luxury Goods', 29)
('Electrical Equipment', 28)
('Electric Utilities', 28)
('Trading Companies & Distributors', 25)
('Building Products', 23)
('Road & Rail', 22)
('Real Esta

In [261]:
top_sectors = [
'Banks',
'Biotechnology',
'Oil, Gas & Consumable Fuels',
'Equity Real Estate Investment Trusts ',
'Metals & Mining',
'Machinery',
'Electronic Equipment, Instruments & C',
'Capital Markets',
'Health Care Equipment & Supplies',
'Insurance',
'Semiconductors & Semiconductor Equipment',
'Software',
'Specialty Retail',
'IT Services',
'Commercial Services & Supplies',
'Pharmaceuticals',
'Chemicals',
'Hotels, Restaurants & Leisure',
'Thrifts & Mortgage Finance',
'Communications Equipment',
'Health Care Providers & Services',
'Food Products',
'Media',
'Aerospace & Defense',
'Household Durables',
'Professional Services',
'Energy Equipment & Services',
'Blank Check / SPAC',
'Electrical Equipment',
'Textiles, Apparel & Luxury Goods',
'Electric Utilities',
'Trading Companies & Distributors',
'Real Estate Management & Development',
'Building Products',
'Entertainment',
'Technology Hardware, Storage & Periph',
'Construction & Engineering',
'Road & Rail',
'Consumer Finance',
'Beverages',
'Auto Components',
'Diversified Telecommunication Services',
'Life Sciences Tools & Services',
'Multi-Utilities',
'Internet & Direct Marketing Retail',
'Personal Products',
'Diversified Consumer Services',
'Mortgage Real Estate Investment Trust',
'Shell Companies',
'Marine',
'Airlines',
'Food & Staples Retailing',
'Wireless Telecommunication Services',
'Containers & Packaging',
'Software-Application',
'Water Utilities',
'Gas Utilities',
'Air Freight & Logistics',
'Leisure Products',
'Health Care Technology',
'Interactive Media & Services',
'Household Products',
'Automobiles',
'Construction Materials',
'Multiline Retail',
'Independent Power and Renewable Elect',
'Paper & Forest Products',
'Diversified Financial Services',
'Industrial Conglomerates',
'Distributors',
'Medical Instruments & Supplies',
'Tobacco',
'Information Technology Services',
'Medical Devices',
'Drug Manufacturers-Specialty & Generic',
'Asset Management',
'Specialty Industrial Machinery',
'Telecom Services',
'Furnishings, Fixtures & Appliances',
'Banks-Regional',
'Transportation Infrastructure',
'Health Information Services',
'Electrical Equipment & Parts',
'Other Industrial Metals & Mining',
'Scientific & Technical Instruments',
'Gold',
'Security & Protection Services',
'Auto Parts',
'Gambling',
'Communication Equipment',
'Software-Infrastructure',
'Health Care',
'Medical Care Facilities',
'Computer Hardware',
'Building Products & Equipment',
'Auto Manufacturers',
'Electronic Components',
'Agricultural Inputs',
'Internet Content & Information',
'Securities and Commodity Exchanges',
'Farm Products',
'Utilities-Renewable',
'Industrial Distribution',
'Electronic Gaming & Multimedia',
'Education & Training Services',
'Pharmaceutical Retailers',
'Oil & Gas Drilling',
'Banks-Diversified',
'Internet Retail',
'Rental & Leasing Services',
'Staffing & Employment Services',
'Auto & Truck Dealerships',
'Electromedical and Electrotherapeutic',
'REIT-Mortgage',
'Petroleum Refineries',
'Advertising Agencies',
'Diagnostics & Research',
'Specialty Business Services',
'Uranium',
'Engineering & Construction',
'Packaged Foods',
'REIT-Residential',
'Beverages-Wineries & Distilleries',
'Semiconductors',
]

### Getting Stocks by sector
After we have obtained the top sectors, the snippet of code below goes through the data and finds all the stocks corresponding to our top sectors

In [262]:
stocks_by_sector = []
for sector in top_sectors:
    stocks = df_total.loc[df_total['Sector'] == sector]
    value = (sector,stocks)
    stocks_by_sector.append(value)

### Calculating Average Intra-Sector Correlation
The following code blocks calculate the average weight of a link between all the stocks in a given sector. For example, in the Bank sector, the functions will go and obtain the links from one bank stock to every other bank stock, and add them to the total. Then this total will be divided by the number of links, to obtain the average weight. This is in hopes of answering our third and fourth research questions.

In [263]:
def calculateIntraSectorWeight(sector):
    max_corr = (0,"stock","stock")
    min_corr = (0,"stock","stock")
    sum_of_links = 0
    link_count = 0
    sector_index = top_sectors.index(sector)
    df = stocks_by_sector[sector_index][1]
    num_of_stocks = df.shape[0]
    for i in tqdm(range(num_of_stocks)):
        first_stock = df.iloc[i,2:]
        for j in range(i+1,num_of_stocks):
            second_stock = df.iloc[j,2:]
            result = 1-spatial.distance.cosine(first_stock,second_stock)
            if(result>max_corr[0]):
                max_corr = (result,df.iloc[i,0],df.iloc[j,0])
            if(result<min_corr[0]):
                min_corr = (result,df.iloc[i,0],df.iloc[j,0])
            sum_of_links += result
            link_count += 1
    if(link_count < 1):
        return 0,1,"N/A","N/A"
    average_link = sum_of_links / link_count
    return average_link,num_of_stocks,max_corr,min_corr

In [264]:
def getAllIntraSectorCorrelations(df):
    for count,sector in enumerate(top_sectors):
        avg,num_stocks,max_corr, min_corr = calculateIntraSectorWeight(sector)
        df.loc[count] = [(str(sector)+"("+str(num_stocks)+")"),
                         avg,
                         (max_corr[0],max_corr[1],max_corr[2]),
                         (min_corr[0],min_corr[1],min_corr[2])]

In [265]:
df_intra = pd.DataFrame(columns=["Sector","Average Correlation","Max Correlation","Min Correlation"])
getAllIntraSectorCorrelations(df_intra)

  0%|          | 0/246 [00:00<?, ?it/s]

  0%|          | 0/126 [00:00<?, ?it/s]

  0%|          | 0/122 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/89 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

#### Filtering & Exporting the Data to Excel

In [266]:
def intraThresh(sector, corr):
    stocks = int(re.sub("[^0-9]", "",sector))
    return stocks*corr

In [267]:
for index, row in df_intra.iterrows():
    total_stocks = intraThresh(row['Sector'],row['Average Correlation'])
    if(total_stocks < 2.5):
        df_intra.drop(index, inplace=True)

In [268]:
df_intra = df_intra.sort_values('Average Correlation', ascending=False)
df_intra

,Sector,Average Correlation,Max Correlation,Min Correlation
43,Multi-Utilities(18),0.540965,"(0.9085339576171104, CMS, WEC)","(0, stock, stock)"
53,Containers & Packaging(12),0.486125,"(0.73334832175076, IP, PKG)","(0, stock, stock)"
26,Energy Equipment & Services(32),0.485446,"(0.8882941728141304, HAL, SLB)","(0, stock, stock)"
68,Industrial Conglomerates(6),0.480197,"(0.7423969814905563, HON, ROP)","(0, stock, stock)"
37,Road & Rail(22),0.478792,"(0.829895696574278, CSX, NSC)","(0, stock, stock)"
50,Airlines(14),0.450195,"(0.8928999467636979, CEA, ZNH)","(0, stock, stock)"
63,Construction Materials(8),0.448190,"(0.8092591332032857, MLM, VMC)","(0, stock, stock)"
30,Electric Utilities(28),0.429635,"(0.8963693868337042, LNT, XEL)","(-0.12298661607003947, EDN, FE)"
40,Auto Components(21),0.418966,"(0.7883215563934879, BWA, DAN)","(0, stock, stock)"
33,Building Products(23),0.410096,"(0.7005375149402394, LII, MAS)","(-0.08748488327308834, AOS, APT)"


In [269]:
df_intra.to_excel("data/sector_correlations/total_intra.xlsx",sheet_name="Sheet1",index=False)

### Calculating Average Intra-Sector Correlation
The next few blocks of code perform a similar calculation as above, however this time we are comparing the stocks in different sectors to eachother, rather than looking within the same sector.

In [270]:
def calculateIntraSectorCorrelations(sector1,sector2):
    sector1_index = top_sectors.index(sector1)
    df1 = stocks_by_sector[sector1_index][1]
    sector1_stock_count = df1.shape[0]
    
    sector2_index = top_sectors.index(sector2)
    df2 = stocks_by_sector[sector2_index][1]
    sector2_stock_count = df2.shape[0]
    
    link_count = 0
    sum_of_links = 0
    max_corr = (0,"stock","stock")
    min_corr = (0,"stock","stock")
    for i in range(sector1_stock_count):
        first_stock = df1.iloc[i,2:]
        for j in range(sector2_stock_count):
            second_stock = df2.iloc[j,2:]
            result = 1-spatial.distance.cosine(first_stock,second_stock)
            if(result>max_corr[0]):
                max_corr = (result,df1.iloc[i,0],df2.iloc[j,0])
            if(result<min_corr[0]):
                min_corr = (result,df1.iloc[i,0],df2.iloc[j,0])
            sum_of_links += result
            link_count += 1
            
    if(link_count < 1):
        return 0,1,1,"N/A","N/A"
    average_link = sum_of_links / link_count
    
    return average_link, sector1_stock_count, sector2_stock_count, max_corr, min_corr

In [271]:
def getAllInterSectorCorrelations(df):
    num_of_sectors = len(top_sectors)
    count = 1
    for i in tqdm(range(num_of_sectors)):
        sector1 = top_sectors[i]
        for j in range(i+1,num_of_sectors):
            sector2 = top_sectors[j]
            avg_sector_corr, sector1_stock_count, sector2_stock_count, max_corr, min_corr = calculateIntraSectorCorrelations(sector1,sector2)
            df.loc[count] = [(str(sector1)+"("+str(sector1_stock_count)+")"),
                             (str(sector2)+"("+str(sector2_stock_count)+")"),
                             avg_sector_corr,
                             (max_corr[0],max_corr[1],max_corr[2]),
                             (min_corr[0],min_corr[1],min_corr[2])]
            count += 1

### Thresholding and exporting Data
Due to the large number of sectors with single stocks, or small stock counts, we need to remove correlations that are  on a small number of nodes (<20).

In [272]:
df_inter = pd.DataFrame(columns=["Sector1","Sector2","Average Correlation","Max Correlation","Min Correlation"])
getAllInterSectorCorrelations(df_inter)

  0%|          | 0/124 [00:00<?, ?it/s]

In [273]:
def threshHold(sector1, sector2):
    stocks1 = int(re.sub("[^0-9]", "",sector1))
    stocks2 = int(re.sub("[^0-9]", "",sector2))
    if stocks1 == 1 or stocks2 == 1:
        return 0
    return stocks1+stocks2

In [274]:
for index, row in df_inter.iterrows():
    total_stocks = threshHold(row['Sector1'],row['Sector2'])
    if(total_stocks < 20):
        df_inter.drop(index, inplace=True)

In [275]:
df_inter = df_inter.sort_values('Average Correlation',ascending=False)
df_inter

,Sector1,Sector2,Average Correlation,Max Correlation,Min Correlation
3268,Electric Utilities(28),Multi-Utilities(18),0.476321,"(0.9167574662755461, XEL, WEC)","(-0.12302158185388956, EDN, WEC)"
2922,Energy Equipment & Services(32),Asset Management(3),0.435951,"(0.7567211391850873, CLB, TYG)","(0, stock, stock)"
668,Machinery(85),Industrial Conglomerates(6),0.429769,"(0.8047939223090763, PH, HON)","(-0.014665736301675336, BIMI, ROP)"
3916,Road & Rail(22),Industrial Conglomerates(6),0.424758,"(0.6942029569397065, UNP, HON)","(0, stock, stock)"
678,Machinery(85),"Furnishings, Fixtures & Appliances(2)",0.419180,"(0.5981576249877013, PH, MLKN)","(-0.057169176359127105, BIMI, MLKN)"
...,...,...,...,...,...
4593,Personal Products(19),Computer Hardware(2),-0.006499,"(0.23413969902843723, VERU, RCAT)","(-0.2188329995059619, MED, RCAT)"
4275,Diversified Telecommunication Services(20),Computer Hardware(2),-0.013513,"(0.29385908454935894, CUEN, RCAT)","(-0.2557492081702679, ATNI, RCAT)"
2745,Household Durables(39),Computer Hardware(2),-0.016439,"(0.22119097939447618, PFIN, RCAT)","(-0.1840168627878105, MDC, RCAT)"
3318,Electric Utilities(28),Computer Hardware(2),-0.048941,"(0.10404706407301423, PCG, RCAT)","(-0.18781927658424413, FTS, QUBT)"


In [276]:
df_inter.to_excel("data/sector_correlations/total_inter.xlsx",sheet_name="Sheet1",index=False)

### Repeating the Process for yearly data
Now that we have found the intra sector correlations for the total data, we can apply the same idea to the yearly data to find trends over time.

In [277]:
def calculateIntraSectorWeightYearly(sector,start_col):
    max_corr = (0,"stock","stock")
    min_corr = (0,"stock","stock")
    sum_of_links = 0
    link_count = 0
    sector_index = top_sectors.index(sector)
    df = stocks_by_sector[sector_index][1]
    df = df.iloc[:,start_col:start_col+12]
    num_of_stocks = df.shape[0]
    for i in range(num_of_stocks):
        first_stock = df.iloc[i,2:]
        for j in range(i+1,num_of_stocks):
            second_stock = df.iloc[j,2:]
            result = 1-spatial.distance.cosine(first_stock,second_stock)
            if(result>max_corr[0]):
                max_corr = (result,df.iloc[i,0],df.iloc[j,0])
            if(result<min_corr[0]):
                min_corr = (result,df.iloc[i,0],df.iloc[j,0])
            sum_of_links += result
            link_count += 1
    if(link_count < 1):
        return 0,1
    average_link = sum_of_links / link_count
    
    return average_link,num_of_stocks

In [278]:
def getAllIntraSectorCorrelationsYearly(df,start_col):
    for count,sector in enumerate(top_sectors):
        avg,num_stocks = calculateIntraSectorWeightYearly(sector,start_col)
        df.loc[count] = [(str(sector)+"("+str(num_stocks)+")"),avg]

In [279]:
def filterIntra(df_intra):
    for index, row in df_intra.iterrows():
        total_stocks = intraThresh(row['Sector'],row['Average Correlation'])
        if(total_stocks < 4):
            df_intra.drop(index, inplace=True)

In [ ]:
start_col = 2
for year in tqdm(range(2010,2020)):
    # Setup and file names
    intra_file_name = "data/sector_correlations/"+str(year)+"_intra.xlsx"
    inter_file_name = "data/sector_correlations/"+str(year)+"_inter.xlsx"
    
    # calculate yearly intra correlation for all sectors + filter
    df_intra_yearly = pd.DataFrame(columns=["Sector","Average Correlation"])
    getAllIntraSectorCorrelationsYearly(df_intra_yearly,start_col)
    filterIntra(df_intra_yearly)
    df_intra_yearly = df_intra_yearly.sort_values('Average Correlation', ascending=False)
    
    # export
    df_intra_yearly.to_excel(intra_file_name,sheet_name="Sheet1",index=False)
    start_col += 12